Obj: Teste de conversão de luminosidade para massa e posterior inclusão no perfil de matéria do Pyautolens

In [20]:
import autolens as al
import autolens.plot as aplt
from pyprojroot import here
import numpy as np

from time import perf_counter as clock

from astropy.cosmology import Planck15 as cosmo
from astropy.constants import G, M_sun, c
import astropy.units as u

workspace_path = str(here())
print("Workspace Path: ", workspace_path)

Workspace Path:  /home/carlos/autolens_workspace


In [21]:
#Lens parameters

z_lens = 0.035
z_source = 2.1

D_l = cosmo.angular_diameter_distance(z_lens)
D_s = cosmo.angular_diameter_distance(z_source)
D_ls = cosmo.angular_diameter_distance_z1z2(z_lens, z_source)

#Useful constants
metre2Mpc = (1*u.m).to(u.Mpc)/u.m           #Constant factor to convert metre to Mpc.
kg2Msun = (1*u.kg/M_sun)*u.solMass/u.kg     #Constant factor to convert kg to Msun

G_Mpc = G*(metre2Mpc)**3/kg2Msun            #Gravitational constant in Mpc³/(Msun s²)
c_Mpc = c*metre2Mpc                         #Speed of light in Mpc/s

In [22]:
#Data from MGE parametrization
surf_star_dat, sigma_star_dat, qstar_dat = np.loadtxt('/home/carlos/Desktop/Tests JAM- Emcee/JAM Input.txt', unpack=True) #Fotometria
surf_DM_dat, sigma_DM_dat, qDM_dat = np.loadtxt('/home/carlos/Desktop/Tests JAM- Emcee/pseudo-DM Input.txt', unpack=True) #DM

In [23]:
#Convert  pot_DM_dat to total mass per Guassian

surf_DM_dat = surf_DM_dat*(u.solMass/u.pc**2)                                    #Surface Density in M_sun/pc²
sigma_DM_dat_ARC = sigma_DM_dat*u.arcsec                                         #Sigma in arcsec
sigma_DM_dat_PC = (sigma_DM_dat_ARC*D_l).to(u.pc, u.dimensionless_angles())      #Convert sigma in arcsec to sigma in pc
Mass_DM_dat = 2*np.pi*surf_DM_dat*(sigma_DM_dat_PC**2)*qDM_dat               #Total mass per gaussian component in M_sun


Mass_DM_dat

<Quantity [1.23194867e-09, 6.68028441e-09, 3.58893618e-08, 1.89897869e-07,
           9.89900450e-07, 5.08804021e-06, 2.58197425e-05, 1.29571456e-04,
           6.44253580e-04, 3.18059614e-03, 1.56249060e-02, 7.65492253e-02,
           3.74810798e-01, 1.83785358e+00, 9.04115642e+00, 4.46886947e+01,
           2.22165111e+02, 1.11133428e+03, 5.59244260e+03, 2.82854563e+04,
           1.43578675e+05, 7.29981748e+05, 3.70825945e+06, 1.87705133e+07,
           9.44300607e+07, 4.74434743e+08, 2.54821917e+09, 1.97116739e+10,
           6.75514574e+11] solMass>

In [24]:
#Convert  surf_star_dat to total Luminosity per Guassian and then to total mass per gaussian

surf_star_dat = surf_star_dat*(u.solLum/u.pc**2)                                    #Surface luminosity Density in L_sun/pc²
sigma_star_dat_ARC = sigma_star_dat*u.arcsec                                        #Sigma in arcsec
sigma_star_dat_PC = (sigma_star_dat_ARC*D_l).to(u.pc, u.dimensionless_angles())     #Convert sigma in arcsec to sigma in pc
Lum_star_dat = 2*np.pi*surf_star_dat*(sigma_star_dat_PC**2)*qstar_dat               #Total luminosity per gaussian component in L_sun

ML = np.array([1,2,3,4,5,6,7])*u.solMass/u.solLum                                   #Mass to light ratio per gaussian in M_sun/L_sun

Mass_star_dat = Lum_star_dat*ML                                                     #Total star mass per gaussian in M_sun
Mass_star_dat

<Quantity [3.95990609e+07, 4.28987743e+08, 7.78636403e+09, 2.39727559e+10,
           7.28088862e+10, 1.47640624e+11, 5.46488824e+11] solMass>

In [38]:
#Defining inputs for the integral below
i = np.deg2rad(85)*u.rad                                           #Inclination angle in rad
Total_Mass = np.append(Mass_star_dat, Mass_DM_dat)                 #Mass per gaussian component in M_sun
Total_q = np.append(qstar_dat, qDM_dat)                            #Total axial ratio per gaussian
Total_q_proj = np.sqrt(Total_q**2 - np.cos(i)**2)/np.sin(i)        #Total projected axial ratio per gaussian
Total_sigma_ARC = np.append(sigma_star_dat_ARC, sigma_DM_dat_ARC)  #Total sigma per gaussian in arcsec
Total_sigma_RAD = Total_sigma_ARC.to(u.rad)                        #Total sigma per gaussian in radians
